<font size="3">First, we import the necessary libraries.</font>

In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn import preprocessing, neighbors
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
import features
#!pip install --user imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
import image_generator
import database

In [6]:
image_generator.main()
imgGen = image_generator.ImageGenerator('images/val/images', 'images/val/val.txt', 'images/val/synset_words.txt')
label_list = imgGen.get_label_list()

images/val/images\ILSVRC2012_val_00018549.JPEG pencil box, pencil case
images/val/images\ILSVRC2012_val_00042874.JPEG scuba diver
images/val/images\ILSVRC2012_val_00024143.JPEG pot, flowerpot
images/val/images\ILSVRC2012_val_00026369.JPEG space bar
images/val/images\ILSVRC2012_val_00012663.JPEG Petri dish
images/val/images\ILSVRC2012_val_00021419.JPEG tobacco shop, tobacconist shop, tobacconist
images/val/images\ILSVRC2012_val_00042860.JPEG Saint Bernard, St Bernard
images/val/images\ILSVRC2012_val_00011560.JPEG viaduct
images/val/images\ILSVRC2012_val_00040447.JPEG water snake
images/val/images\ILSVRC2012_val_00018870.JPEG jackfruit, jak, jack


<font size="5">**Running the premodel**</font>
<br /><br />
<font size="3">We train the premodel with 18k images and receive labels for the 2k test images.</font>

In [7]:
percentage = features.prototype(20000, (['lg', 'lg', 'lg'], ['lg', 'lg', 'lg']))          # using logistic regression for the premodel

<font size="5">**Testing the accuracy of the premodel**</font>
<br /><br />
<font size="3">First, we extract the predicted models for the 2k images.</font>

In [9]:
def img_num_from_name(img_name):          # obtaining the image number from image name
    ext = []
    for i in range(len(img_name)):
        if i > 14 and i < 23:
            ext.append(img_name[i])
    j = 0
    for i in range(len(ext)):
        if ext[i] == '0':
            j = j + 1
        if ext[i] != '0':
            break
    rightNum = []
    for i in range(j, len(ext)):
        rightNum.append(ext[i])
    rightNum = ''.join(rightNum)
    rightNum = int(rightNum)
    return rightNum


predicted_models = []
for i in range(len(percentage[4][0])):
    img_paths = (percentage[4][0][i][0])
    img_nums = (img_num_from_name(img_paths))
    if percentage[4][0][i][4] == 'tf-mobilenet_v1' and percentage[4][0][i][3] == 1:
        predicted_models.append([img_nums, 'mobilenet_v1'])
    elif percentage[4][0][i][4] == 'tf-inception_v4' and percentage[4][0][i][3] == 2:
        predicted_models.append([img_nums, 'inception_v4'])
    elif percentage[4][0][i][4] == 'tf-resnet_v1_152' and percentage[4][0][i][3] == 3:
        predicted_models.append([img_nums, 'resnet_v1_152'])
    elif percentage[4][0][i][4] == 'failed':
        predicted_models.append([img_nums, 'failed'])

<font size="3">Now, we obtain the accuracy results from the database.</font>

In [11]:
accuracy_premodel = 0
for i in range(len(predicted_models)):
    if predicted_models[i][1] != 'failed':
        accuracy_premodel = accuracy_premodel + (database.get_model_top_1("inference", predicted_models[i][0], predicted_models[i][1]))
print("Premodel accuracy = {}".format(accuracy_premodel/len(predicted_models)))

Premodel accuracy = 0.742


<font size="5">**Comparing the results to MobileNet, Inception and ResNet**</font>
<br /><br />
<font size="3">We start by getting the image numbers of the 2k test images.</font>

In [14]:
img_nums = []
img_paths = []
for i in range(len(percentage[4][0])):
    img_paths.append(percentage[4][0][i][0])
img_nums = []
for i in range(len(img_paths)):
    img_nums.append(img_num_from_name(img_paths[i]))

len(img_nums) == len(set(img_nums))           # making sure there are no duplicates

#results = []
#img_path_list = []
#img_labels = []
#for i in range(len(img_nums)):
    #img_path, img_label = imgGen.get_image_data(img_nums[i])
    #img_path_list.append(img_path)
    #img_labels.append(img_label)
#print(len(img_labels))

True

<font size="3">Finally, we obtain the accuracy results from the database.</font>

In [15]:
mobilenet_right = 0
for i in range(len(img_nums)):
    mobilenet_right = mobilenet_right + (database.get_model_top_1("inference", img_nums[i], 'mobilenet_v1'))
print("Mobilenet top 1 accuracy = {}".format(mobilenet_right/len(img_nums)))

inception_right = 0
for i in range(len(img_nums)):
    inception_right = inception_right + (database.get_model_top_1("inference", img_nums[i], 'inception_v4'))
print("Inception top 1 accuracy = {}".format(inception_right/len(img_nums)))

resnet_right = 0
for i in range(len(img_nums)):
    resnet_right = resnet_right + (database.get_model_top_1("inference", img_nums[i], 'resnet_v1_152'))
print("ResNet top 1 accuracy = {}".format(resnet_right/len(img_nums)))

Mobilenet top 1 accuracy = 0.7105
Inception top 1 accuracy = 0.7995
ResNet top 1 accuracy = 0.762
